# Importando as bibliotecas necessárias

In [ ]:
const faker = require('faker');
const mongoose = require('mongoose');
mongoose.set('useCreateIndex', true);
const _ = require('lodash');
const countryData = require('country-data');
console.log('Done!');

# Conectando a base de dados usando mongoose

In [ ]:
mongoose.connect('mongodb://localhost/effective_store', { useUnifiedTopology: true, useNewUrlParser: true }).then(() => console.log('Connected'));
console.log('Connecting...')

# Definindo os países disponíveis

In [ ]:
const availableCountries = _.filter(countryData.countries.all, countrie => countrie.status === 'assigned');
console.log('Done!');

# Definindo o schema de Invoice

In [ ]:
const InvoiceSchema = new mongoose.Schema({
  invoiceNumber: {
    type: Number,
    unique: true
  },
  dueDate: Date,
  status: {
    type: String,
    enum: ['Payed', 'Waiting Payment', 'Canceled']
  },
  totalAmount: Number,
  invoiceDoc: [{
    sellerCode: {
      type: String,
      unique: true
    },
    docNumber: String,
    productList: [{
      name: String,
      price: Number
    }]
  }]
})
console.log('Done!');

# Definindo o model de Invoice

In [ ]:
const Invoice = mongoose.model('Invoice', InvoiceSchema);
console.log('Done!');

# Definindo o schema de Sellers

In [ ]:
const SellerSchema = new mongoose.Schema({
  code: {
    type: Number,
    unique: true
  },
  name: String,
  country: {
    code: { type: String, enum: availableCountries.map(country => country.alpha2) },
    name: { type: String, enum: availableCountries.map(country => country.name) },
    emojiFlag: { type: String, enum: availableCountries.map(country => country.emoji) }
  },
  invoices: [InvoiceSchema],
  createdAt: Date,
  productCatalog: [{
    name: String,
    price: Number
  }]
});
console.log('Done!');

# Definindo o model de Seller

In [ ]:
const Seller = mongoose.model('Seller', SellerSchema);
console.log('Done!');

# Gerando dados de sellers

In [ ]:
const SELLERS_LENGTH = 500;
const PRODUCTS_LENGTH = 100;

const availableProducts = _.range(PRODUCTS_LENGTH).map(() => ({
  name: faker.commerce.product(),
  price: parseFloat(faker.commerce.price()),
  description: faker.commerce.productDescription(),
}));

const sellers = _.range(SELLERS_LENGTH).map((sellerIndex) => {
  const productCatalogSize = _.random(1, PRODUCTS_LENGTH)
  const country = _.sample(availableCountries)
  const productCatalog = []
  _.range(productCatalogSize).forEach(() => {
    const productSample = _.sample(availableProducts)
    if (!productCatalog.includes(productSample)) {
      productCatalog.push(productSample)
    }
  });
  
  return new Seller({
    code: sellerIndex,
    name: faker.company.companyName(),
    country: { code: country.alpha2, name: country.name, emojiFlag: country.emoji },
    createdAt: faker.date.past(10),
    productCatalog
  });
});
console.log('Done!');

# Gerando dados de invoices

# Populando base de dados
Limpamos a collection de sellers e após isto inserimos os dados gerados

In [ ]:
console.log('Cleaning sellers collection');
Seller.deleteMany({}, () => console.log('cleared'));
console.log('Inserting generated data');
Seller.insertMany(sellers, () => console.log('Inserted'));
console.log('Processing...');

# Query 1 - Lojistas (Sellers) com maior número de produtos disponíveis na plataforma

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      productCatalogSize: { $cond: { if: { $isArray: "$productCatalog" }, then: { $size: "$productCatalog" }, else: "NA"} }
    }
  },
  {
    $sort : { productCatalogSize : -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 2 - Produtos mais comuns entre os lojistas associados

In [ ]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 3 - Países com mais lojistas

In [ ]:
Seller.aggregate([
  {
    $group: {
      _id: '$country.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 4 - Produtos com mais lojistas por região

In [ ]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      country: { $first : "$country.name" },
      counts: { $sum: 1 },
    }
  },
  {
    $sort: { country: -1 }
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 5 - Lista dos lojistas (Sellers) ordenada pelos mais novos

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 6 - Lista dos lojistas (Sellers) ordenada pelos mais antiga

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: 1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 7 - Lista das faturas que estão em aberto para os lojistas(Sellers) com menos de 1 ano de mercado

# Query 8 - Lista das faturas acima de 1000,00